In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import snowFun
import dask.array
# %matplotlib widget

# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')


In [ ]:
# open wolverine data 
file_name = os.path.join(folder_AGVA, 'coarsen', "original.tif")
xr_original = riox.open_rasterio(file_name)

In [ ]:
xr_original.head()

In [ ]:
# coarsen and save
xr_coarse = xr_original.coarsen({"x":3, "y":3}, boundary="trim").median(skipna=True)

In [ ]:
path_save = os.path.join(folder_AGVA, 'coarsen', "coarse3.tif")
xr_coarse.rio.to_raster(path_save)

In [2]:
folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')
path_test = os.path.join(folder_save, 'Daily AAs', f"S2_RGI60-01.14443_2018_daily_AAs.nc")  

# open data
xr_to_use = xr.open_dataset(path_test, chunks={'band':1, 'y':-1, 'x':-1})['class']#.to_array()
# print(xr_to_use.head)

# coarsen and save
usable = xr.where(xr_to_use>0, 1, 0).astype("uint8").coarsen({"x":5, "y":5}, boundary="trim").mean(skipna=True)
print(usable.dims)

# save
usable.rio.to_raster(os.path.join(folder_save, 'temp.tif'), dtype='uint8')

percent_usable_by_time = (usable.sum(dim=['x','y'])*25/10287990).compute().values[0]

('time', 'y', 'x')


In [17]:
path_temp = os.path.join(folder_save, 'temp.tif')
usable_temp = riox.open_rasterio(path_temp)
percent_usable_by_time = (usable_temp.sum(dim=['x','y'])).compute()

In [26]:
percents = percent_usable_by_time.values*25*100/(2834.493*1000*1000)*100
print(percents.astype(int))

[ 0  9  0 30 85  0  1  0 27 76 90  0  2 59  7  9  1 77  0 31 17 21  3 51
 28 16 40  7  6 32 32  5  1 23  1  5 90 45 79 47  2  4 32  0  1 37 23 79
 89 49 64  8 21 78 89  1  0  0  0  3 27  2  6 11 14 31  3  4  9 13  2 57
 72 35 77 59 42 26 86  4 38 45 72 83 45  6  0  2 69 33 68 35 21 51  0 30
  0  0  6  1 10  0  0  0  0 44 21 12 15  4 31 25 33 10 25  0 19  0  0 34
  0  4  0  0]
